In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def gen_serve_dist_map(file_path, player1):
    events = pd.read_csv(file_path)
    events['pointWonBy'] = events.groupby('pointNumber')['pointWonBy'].bfill()
    
    filtered_events = events[
        (events['shotInRally'] == 1) &
        (events['serverName'] == player1) &
        ((events['firstServeIn'] == 1.0) | (events['secondServeIn'] == 1.0))
    ]
    
    filtered_events['Zone'] =  filtered_events['side'] + " " + filtered_events['serveInPlacement']

    serve_counts = filtered_events.groupby('Zone').size()
    
    won_counts = filtered_events[filtered_events['pointWonBy'] == player1].groupby('Zone').size()
    won_counts = won_counts.reindex(serve_counts.index, fill_value=0)
    
    serve_dist = pd.DataFrame({
        "Zone": serve_counts.index,
        "Win Proportion": won_counts.astype(str) + '/' + serve_counts.astype(str)
    }).reset_index(drop=True)

    serve_dist['Server'] = [player1, "", "", "", "", ""]
    
    serve_dist_map_json = serve_dist.to_json(orient='records')
    with open('serve_dist_map.json', 'w') as f:
        f.write(serve_dist_map_json)

/var/folders/gc/y_rdwxcs2pb0n4dz4z33dcth0000gn/T/ipykernel_12426/3658812643.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
file_path = "../../Shot_Visuals_EliseWagle_MotkaKomac.csv"
player1 = "Elise Wagle"
gen_serve_dist_map(file_path, player1)
